# PYNQ 에서 AXI GPIO 사용하기

## 목표
이 노트북을 통해 PYNQ에서 AXI GPIO 를 사용하는 것을 실습해보고자 한다.

AXI GPIO의 특징은 다수의 AXI GPIO 컨트롤러를 프로그래밍 가능한 로직에 적용할 수 있다는 점이다. 이를 통해 내부 또는 외부의 GPIO 신호를 제어할 수 있다.

## 하드웨어 디자인

이 예제에서는 PYNQ-Z1 또는 PYNQ-Z2 보드에 탑재된 LED와 버튼, 스위치에 연결돤 3개의 AXI GPIO 컨트롤러를 사용한다. (각 AXI GPIO 컨트롤러는 2개의 채널을 가지고 있다. 따라서 하나의 AXI GPIO IP를 통해 여러 주변 시스템들을 제어하는 것이 가능하다. 그러나 예제에서는 단순함과 시연을 위해 AXI GPIO 컨트롤러를 분리해서 사용하였다.) 

![AXI GPIO Design](./images/axi_gpio_design.png "AXI GPIO Design")

### 1. 튜토리얼 Overlay 다운로드 하기

`axi_gpio.bit` 파일과 `axi_gpio.tcl` 파일은 현재 노트가 위치한 폴더 내의 `/bitstream` 폴더에 내에서 발견할 수 있다. bitsteam 파일은 Overlay 클래스에게 상대 주소를 전달함으로써 루트 라이브러리 폴더에 위치하지 않아도 사용할 수 있다.

* bitstream 폴더 내에 bitstream 파일과 .tcl 파일 확인하기

In [17]:
!dir ./bitstream

axi_gpio.bit  ps_gpio.bit  pynq_tutorial.bit  resize.bit
axi_gpio.tcl  ps_gpio.tcl  pynq_tutorial.tcl  resize.tcl


* Overlay에 bitstream 적용하기

In [18]:
from pynq import Overlay
axi_gpio_design = Overlay("./bitstream/axi_gpio.bit")

/usr/local/lib/python3.6/dist-packages/pynq/overlay.py:299: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


다운로드하고자 하는 IP Dictionary의 디자인을 확인한다. IP Dictionary는 디자인 내의 AXI IP를 나열하며, 이 예에서는 버튼과 LED, 그리고 스위치에 대한 AXI GPIO 컨트롤러를 나열한다. 물리적 주소와, 주소의 범위 및 IP 유형이 나열되는 것 또한 볼 수 있다. 만약 어떤 인터럽트가 발생하거나, GPIO가 PS에 연결되어 있다면, 이러한 사항들 또한 출력된다.

In [19]:
axi_gpio_design.ip_dict

{'buttons': {'addr_range': 65536,
  'driver': pynq.lib.axigpio.AxiGPIO,
  'fullpath': 'buttons',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'SEG_buttons_Reg',
  'phys_addr': 1092681728,
  'state': None,
  'type': 'xilinx.com:ip:axi_gpio:2.0'},
 'leds': {'addr_range': 65536,
  'driver': pynq.lib.axigpio.AxiGPIO,
  'fullpath': 'leds',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'SEG_leds_Reg',
  'phys_addr': 1092616192,
  'state': None,
  'type': 'xilinx.com:ip:axi_gpio:2.0'},
 'switches': {'addr_range': 65536,
  'driver': pynq.lib.axigpio.AxiGPIO,
  'fullpath': 'switches',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'SEG_switches_Reg',
  'phys_addr': 1092747264,
  'state': None,
  'type': 'xilinx.com:ip:axi_gpio:2.0'}}

## AxiGPIO 클래스

PYNQ의 AxiGPIO 클래스는 AXI GPIO 컨트롤러에 접근하는데 사용된다.

### 1. 스위치와 버튼 제어하기

인스턴스는 IP dictionary 를 통해 찾을 수 있으며, 아래 코드와 같이 변수 선언에도 사용할 수 있다.

In [20]:
from pynq.lib import AxiGPIO

buttons_instance = axi_gpio_design.ip_dict['buttons']
buttons = AxiGPIO(buttons_instance).channel1

In [21]:
buttons.read()

0

버튼 컨트롤러는 PYNQ 보드의 내장된 4개의 버튼(BTN0 부터 BTN3까지) 연결되어 있다. 각 버튼들을 하나 혹은 여러 개를 동시에 눌러 보면서 위의 코드를 재실행해보고 결과가 어떻게 다르게 나타나는지 확인해보자.

스위치를 위한 AXI GPIO 컨트롤러 또한 버튼을 이용한 것과 비슷하게 진행할 수 있다.

In [22]:
switches_instance = axi_gpio_design.ip_dict['switches']
switches = AxiGPIO(switches_instance).channel1

In [23]:
print(f"Switches: {switches.read()}")

Switches: 0


### 2. LED 제어하기
LED 또한 버튼과 스위치에서 한 것과 같이 비슷하게 진행할 수 있다.

In [24]:
from pynq.lib import AxiGPIO
led_instance = axi_gpio_design.ip_dict['leds']
led = AxiGPIO(led_instance).channel1

출력을 슬라이스에 연결할 수 있다.

In [25]:
led[0:4].write(0x1)

In [26]:
from time import sleep

led[0:4].write(0x3)
sleep(1)
led[0:4].write(0x7)
sleep(1)
led[0:4].write(0xf)

* LED 리셋하기

In [27]:
led[0:4].off()

### 3 버튼과 스위치, LED를 한 번에 적용하기

버튼의 입력을 LED로 출력하는 코드를 반복문으로 실행한다.

반복문의 조건은 스위치로 설정하였기 때문에, SW0 스위치가 켜짐 상태에 있어야 아래 코드가 제대로 동작하는 것을 볼 수 있다. 반복문이 실행되는 동안 버튼을 누르면 같은 자리에 위치한 LED의 불이 켜지는 것을 알 수 있다. 반복문 코드를 종료시키기 위해서는 SW0 스위치를 꺼짐 상태로 두면 된다.

In [28]:
while(switches.read() is 1):
    led[0:4].write(buttons.read())